#### This notebook is to *numerically* calculate multitude of random distribution (m.r.d.) expected for $(\phi,\psi)$ diffraction orientations

- Dependents

In [ ]:
pwd

In [ ]:
%pylab inline

import TX, os, shutil
from TX import cmb, euler
euler = euler.euler
path_home=os.getcwd()
# cmb.random

path_load='/Users/yj/repo/rs_pack/src/src_fortran/'
os.chdir(path_load)
import cs
reload(cs)
os.chdir(path_home)

In [ ]:
### Define random orientation
grains=cmb.random(ngrain=10000,phi1=360, phi2=90, phi=360)
print grains.shape

In [ ]:
## rotation matrices for indivial discrete orientations
a_mat = []
ag    = []
for i in xrange(len(grains)):
    gr=grains[i]
    ph,th,tm = gr[:3]
    a=euler(ph=grains[i][0], th=grains[i][1], tm=grains[i][2],echo=False)
    a_mat.append(a)
    ag.append(a.T)

In [ ]:
hkls, n = cs.cubic_eqvect(v=[2,1,1])
hkls    = hkls.T
hkls    = hkls[:n,:]

In [ ]:
print cs.vector_ang.__doc__

In [ ]:
def vecply(a,b):
    c = np.zeros((3,))
    for i in xrange(3):
        for j in xrange(3):
            c[i] = c[i] + b[i][j] * a[j]
    return c

def vnorm(vec):
    v=vec[::]
    mag=0.
    for i in xrange(len(v)):
        mag = mag + v[i]**2
    mag = np.sqrt(mag)
    
    for i in xrange(len(v)):
        v[i] = v[i] / mag
        
    return v

def anorm(v):
    """
    Return "magnitude" of vector <v>
    """
    norm = 0.
    for i in xrange(len(v)):
        norm=norm+v[i]**2
    return np.sqrt(norm)

def angv(v1,v2):
    """
    Calculates angle between two vectors
    """
    dotp = 0.
    for i in xrange(len(v1)):
        dotp = dotp + v1[i] * v2[i]
        
    v1n = anorm(v1)
    v2n = anorm(v2)
    return np.arccos(dotp / v1n/ v2n)

def isclose(v1, v2, th):
    pi = np.pi
    tha = angv(v1, v2) * 180./pi
    
    if tha>90.:  tha = 180. - tha
        
    if tha<=th: return True
    else: return False

In [ ]:
angv([0,0,1],[1,0,0])*180/np.pi

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(10,10))
ax  = fig.add_subplot(221, projection='3d')
ax1 = fig.add_subplot(222)

## phi / psi
phis  = [  0, 45, 90]
betas = [-45,-15, -7,  0,  7, 15, 45]

## angmisdet 
angmisdet = 23.59 ## -- depend on 'hkl' and X-ray tube
angcone   = 10.

x0_R_s      = np.zeros((3,3))
x0_R_s[0,1] = 1.
x0_R_s[1,0] = 1.
x0_R_s[2,2] =-1.

bv0    = np.zeros((3,))
bv0[2] = 1.0  # [0,0,1] 
x      = np.zeros((3,))  # x axis of X-ray coordinate
x[0]   = 1.0

## R p1 in xa <- beam in xa
## R p2 in xa <- beam in xa
p1Rb   = cs.vector_ang(x, +angmisdet/2.)
p2Rb   = cs.vector_ang(x, -angmisdet/2.)
nphi = len(phis)
nbet = len(betas)

wgts = np.zeros((2,nphi,nbet))
ngrs = np.zeros((2,nphi,nbet))

det_ps=np.zeros((2,nphi,nbet,3))

for iphi in xrange(nphi):
    for ibeta in xrange(nbet):
        ph = phis[iphi]
        be = betas[ibeta]
        x1_R_x0 = euler(ph=-ph,th=be,tm=0,echo=False)
        bv1     = vecply(bv0,x1_R_x0)        
        x1_R_s  = cs.matply(x1_R_x0, x0_R_s)
        s_R_x1  = x1_R_s.T

        bvs = vecply(bv0, s_R_x1) ## Beam vector in sample axes        
        ps1 = vecply(bv0, p1Rb)
        ps2 = vecply(bv0, p2Rb)        
        ps1 = vecply(ps1, s_R_x1)
        ps2 = vecply(ps2, s_R_x1)
        
        det_ps[0,iphi,ibeta,:]=ps1[::]
        det_ps[1,iphi,ibeta,:]=ps2[::]

        ax.scatter(ps1[0],ps1[1],ps1[2])
        ax.scatter(ps2[0],ps2[1],ps2[2])
        ax1.plot(ps1[0],ps1[1],'k.')
        ax1.plot(ps2[0],ps2[1],'k.')

        n_det1=0
        for i in xrange(len(ag)):
            agx = ag[i]
            for j in xrange(len(hkls)):
                pc     = vnorm(hkls[j])
                hkl_sa = vecply(pc, agx) ## hkl in sample axes
                ps     = hkl_sa[::]
                
                if (isclose(ps,ps1,angcone)):
                    n_det1=n_det1+1
                    wgts[0,iphi,ibeta] = wgts[0,iphi,ibeta] + grains[i][3]
                    ngrs[0,iphi,ibeta] = ngrs[0,iphi,ibeta] + 1
                if (isclose(ps,ps2,angcone)):
                    wgts[1,iphi,ibeta] = wgts[1,iphi,ibeta] + grains[i][3]
                    ngrs[1,iphi,ibeta] = ngrs[1,iphi,ibeta] + 1
ax.set_zlim(0,-1.)

In [ ]:
def mrd(ngr=100,angcone=10.,phi1=360,phi2=90,phi=360):
    grains=cmb.random(ngrain=ngr,phi1=phi1, phi2=phi2, phi=phi)

    ag=[]
    for i in xrange(ngr):
        a=euler(ph=grains[i][0], th=grains[i][1], tm=grains[i][2],echo=False)
        ag.append(a.T)
    ag=np.array(ag)    

    wgts = np.zeros((2,nphi,nbet))
    ngrs = np.zeros((2,nphi,nbet))

    for iphi in xrange(nphi):
        for ibeta in xrange(nbet):        
            ps1=det_ps[0,iphi,ibeta,:]
            ps2=det_ps[1,iphi,ibeta,:]
            
            for i in xrange(len(ag)):
                agx = ag[i]
                for j in xrange(len(hkls)):
                    pc     = vnorm(hkls[j])
                    hkl_sa = vecply(pc, agx) ## hkl in sample axes
                    ps     = hkl_sa[::]
                
                    if (isclose(ps,ps1,angcone)):
                        wgts[0,iphi,ibeta] = wgts[0,iphi,ibeta] + grains[i][3]
                        ngrs[0,iphi,ibeta] = ngrs[0,iphi,ibeta] + 1
                    if (isclose(ps,ps2,angcone)):
                        wgts[1,iphi,ibeta] = wgts[1,iphi,ibeta] + grains[i][3]
                        ngrs[1,iphi,ibeta] = ngrs[1,iphi,ibeta] + 1
    return wgts.mean(), wgts.std()

In [ ]:
angcone=10.
ngrs=[100,200,400,800]
a=[]
s=[]
for i in xrange(len(ngrs)):
    avg, std=mrd(ngr=ngrs[i],angcone=angcone,phi1=360,phi2=360,phi=360)
    a.append(avg)
    s.append(std)
x=np.arange(len(ngrs))+1
gca().errorbar(x,a,yerr=std)
gca().set_xlim(0.,len(x)+1)

- m.r.d: 0.182 for hkl={211} and angcone=10 from 10,000 grains
- m.r.d: 0.0457 for hkl={211} and angcone=5 from 100,000 grains

In [ ]:
angcone=5.
ngrs=[100,1000,10000,100000]
a=[]
s=[]
for i in xrange(len(ngrs)):
    avg, std=mrd(ngr=ngrs[i],angcone=angcone,phi1=360,phi2=360,phi=360)
    a.append(avg)
    s.append(std)
x=np.arange(len(ngrs))+1
gca().errorbar(x,a,yerr=std)
gca().set_xlim(0.,len(x)+1)

In [ ]:
a

### Trend in m.r.d with respect to increase in diffraction window

In [ ]:
angcones=[1,2,3,4,5,6,8,9,10]
ngr=6000
a=[]; s=[]
for i in xrange(len(angcones)):
    avg, std=mrd(ngr=ngr,angcone=angcones[i],phi1=360,phi2=90,phi=90)
    a.append(avg)
    s.append(std)

#x=np.arange(len(angcones))+1

plt.figure(figsize=(3.5,3))
ax=gca()
ax.errorbar(angcones,a,yerr=std)
ax.set_ylim(0.,)

ax.set_xlabel(r'angluar width $\omega$')
ax.set_ylabel(r'Intensity of random distribution $\mathrm{I^{r.d.}}$')

In [ ]:
plt.figure(figsize=(3.5,3))
ax=gca()
ax.errorbar(angcones,a,yerr=std)
ax.set_ylim(0.,)

ax.set_xlabel(r'angluar width $\omega$')
ax.set_ylabel(r'Intensity of random distribution $\mathrm{I^{r.d.}}$')